<a href="https://colab.research.google.com/github/noiroiseauun/MLOverflow/blob/main/NB_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import statements
import csv
import string
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDRegressor
from sklearn.naive_bayes import CategoricalNB
from datetime import datetime, timedelta
import gc
import pandas as pd
import random
import re
from google.colab import drive


In [ ]:
drive.mount('/content/drive')
datapath = '/content/drive/Shared drives/CMPUT 466 - Project/data/'
questionsPath = f"{datapath}noHTMLQuestions.csv"
answersPath = f"{datapath}Answers-Final.csv"
tagsPath = f"{datapath}Tags-Final.csv"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# answerdf = pd.read_csv("/content/drive/Shared drives/CMPUT 466 - Project/data/Answers-Final.csv", encoding="ISO-8859-1")
answerdf = pd.read_csv(answersPath, encoding="ISO-8859-1")
questiondf = pd.read_csv(questionsPath, encoding="ISO-8859-1", header=None)
tagdf = pd.read_csv(tagsPath)

In [ ]:
questionCols = {0:"Id", 1:"OwnerUserId", 2:"CreationDate", 3:"ClosedDate", 4:"Score", 5:"Title", 6:"Body"}
questiondf.rename(mapper=questionCols, axis='columns', inplace=True)

In [ ]:
tagCounts = tagdf["Tag"].value_counts(normalize=False)
tmpDict = tagCounts.to_dict()
tagdf["counts"] = tagdf["Tag"].map(tmpDict)
tagdf = tagdf[tagdf["counts"] > 101180]

In [ ]:
jointdf = pd.merge(tagdf, questiondf, how="left", left_on="Id", right_on="Id")

In [ ]:
SEED = 19
random.seed(SEED)
SAMPLE = 10000

In [ ]:
tagSample = jointdf.sample(SAMPLE)
vectorizer = CountVectorizer(
        lowercase=True, stop_words='english',
        max_df=1.0, min_df=1, max_features=None,  binary=True,
        encoding="ISO-8859-1"
      )

def removeWhitespace(val):
  return re.sub('\s', ' ', val)
# tagSample = jointdf[:10000] # For consistency, don't sample yet
# tagSample["Body"] = tagSample["Body"].apply(removeWhitespace)
body = tagSample["Body"].apply(removeWhitespace).to_numpy()
tag = tagSample["Tag"].to_numpy()
X = vectorizer.fit_transform(body).toarray()
classifier = CategoricalNB()

In [ ]:
spliter = 9000
x_train = X[0:spliter]
x_test = X[spliter:]
y_train = tag[0:spliter]
y_test = tag[spliter:]
print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))


x_train shape: (9000, 86395)
y_train shape: (9000,)
x_test shape: (1000, 86395)
y_test shape: (1000,)


In [ ]:
classifier.fit(x_train, y_train)

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# classifier.score(x_test, y_test)

In [ ]:
failList = list()
missList = list()
count = 0
tmpResultlist = list()
length = len(x_test)
for index in range(length):
  try:
    result = classifier.predict(x_test[index:index+1])
    tmpResultlist.append(result[0])
    if result[0] == y_test[index]: count += 1
    else:
       missList.append(index)
      #  print("result was {} and true was {}".format(result[0], y_test[index]))
  except Exception as e:
    # print("index {} failed: {}".format(index, e))
    failList.append(index)
failed = len(failList)
missed = len(missList)

print("FAILED: {}".format(failed))
print("Correct: {}".format(count/(length-failed)))
print("missed: {}".format(missed/(length-failed)))
# print("count: {}".format(count))
# print(tmpResultlist)

FAILED: 847
Correct: 0.6666666666666666
missed: 0.3333333333333333
